# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 8.7MB/s 


In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.


There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). 

Now it's time to import these stocks to our Jupyter Notebook file.

In [ ]:
stocks=pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [ ]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SBGKD SALALETLR AGEQON)MAS/(N TCE', 'calculationPrice': 'close', 'open': 125.38, 'openTime': 1639504795348, 'openSource': 'lcfifaoi', 'close': 120.25, 'closeTime': 1690829322403, 'closeSource': 'iiafoflc', 'high': 122.08, 'highTime': 1696551228471, 'highSource': 'eacnieeu 5 medti dy1rlp', 'low': 120.75, 'lowTime': 1656614246479, 'lowSource': 'mdilp ee nayri e15teudc', 'latestPrice': 120.58, 'latestSource': 'Close', 'latestTime': 'March 19, 2021', 'latestUpdate': 1690525224860, 'latestVolume': 194717928, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 121.6, 'delayedPriceTime': 1687722444175, 'oddLotDelayedPrice': 122.23, 'oddLotDelayedPriceTime': 1628179611023, 'extendedPrice': 124.92, 'extendedChange': 0.03, 'extendedChangePercent': 0.00026, 'extendedPriceTime': 1623155741351, 'previousClose': 121.45, 'previousVolume': 124512672, 'change': -0.55, 'changePercent': -0.0047, 'vol

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [ ]:
price=data['latestPrice']
market_cap=data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [ ]:
my_columns=['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [ ]:
final_dataframe.append(
    pd.Series([
               symbol,price,market_cap,'N/A'
    ],index=my_columns),
    ignore_index=True
    
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,120.58,2074839468948,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
  api_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(api_url).json()
  final_dataframe=final_dataframe.append(
      pd.Series([
                 stock,data['latestPrice'],data['marketCap'],'N/A'
                 ], index=my_columns),
                 ignore_index=True
      
  )

In [ ]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.69,38162891833,N/A
1,AAL,26.05,16739895559,N/A
2,AAP,191.00,12628450562,N/A
3,AAPL,120.15,2037020264285,N/A
4,ABBV,103.43,184606331674,N/A
...,...,...,...,...
500,YUM,112.28,33833765885,N/A
501,ZBH,162.08,34676386023,N/A
502,ZBRA,477.08,24759732102,N/A
503,ZION,57.34,9627227487,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
def chunks(l, n):   
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [ ]:
symbol_groups=list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0, len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))  #we're taking all elements of symbol_groups[i] and joining them to create a full string seperated by ','
final_dataframe=pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
  batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
  data=requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    final_dataframe=final_dataframe.append(
        pd.Series([
         symbol,
         data[symbol]['quote']['latestPrice'], #We're looking at the json file ( almost same as dictionnaries in python) : we get the data related to chosen symbol , out of that data we get the data related to 'quote', then from that data get the value of 'lastPrice'
         data[symbol]['quote']['marketCap'],
         'N/A'
        ],
        index=my_columns),
        ignore_index=True
    )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.27,37778397571,N/A
1,AAL,25.73,16456452363,N/A
2,AAP,189.00,12397091351,N/A
3,AAPL,120.88,2069638226557,N/A
4,ABBV,105.87,189867910016,N/A
...,...,...,...,...
500,YUM,110.53,33401391837,N/A
501,ZBH,167.29,34318717969,N/A
502,ZBRA,460.15,25433210338,N/A
503,ZION,58.11,9585366968,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [ ]:
portfolio_size=input('Enter the value of your portfolio :')
valid=False
while (valid==False):
  try:
    val=float(portfolio_size)
    valid=True
  except ValueError:
    print("Try again !, please input a valid value ")
    portfolio_size=input('Enter the value of your portfolio :')


Enter the value of your portfolio :10000000


In [ ]:
position_size=val/len(final_dataframe.index) #We're calculating the sum of money that we can spend on each Stock ( all stocks here are equal, we're going to spend the same amount of money on each stock hence " th equal weight spending")
for i in range(0,len(final_dataframe.index)):
  final_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[i,'Stock Price']) # for each stock calculate the number of shares that you can buy

In [ ]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.27,37778397571,155
1,AAL,25.73,16456452363,769
2,AAP,189.00,12397091351,104
3,AAPL,120.88,2069638226557,163
4,ABBV,105.87,189867910016,187
...,...,...,...,...
500,YUM,110.53,33401391837,179
501,ZBH,167.29,34318717969,118
502,ZBRA,460.15,25433210338,43
503,ZION,58.11,9585366968,340


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. 

### Initializing our XlsxWriter Object

In [ ]:
writer=pd.ExcelWriter('recommended_trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color='#0a0a23'
font_color='#ffffff'
string_format=writer.book.add_format(
    {
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)
dollar_format=writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)
integer_format=writer.book.add_format(
    {
        'num_format':'0',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
column_formats={
    'A' :['Ticker',string_format],
    'B' :['Stock Price', dollar_format],
    'C' :['Market Capitalization', dollar_format],
    'D' :['Number of Shares to Buy',integer_format]
}

for column in column_formats.keys():
  writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',20,column_formats[column][1])
  writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()